<a href="https://colab.research.google.com/github/condehub5/Trabajo-Final---Universidad-del-Pac-fico/blob/main/Trabajo_Final_Estimaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🐟 Proyecto Final: Pronóstico de la Demanda de Pesca para Consumo Interno Nacional en el Perú (1991–2023)

Este proyecto tiene como objetivo estimar y proyectar la **demanda mensual de pesca destinada al consumo interno nacional** en el Perú, utilizando datos oficiales proporcionados por el **Instituto Nacional de Estadística e Informática (INEI)**. La serie abarca desde enero de 1991 hasta diciembre de 2023 y representa una variable relevante para el análisis sectorial de alimentos, producción primaria y consumo nacional.

En contextos de inflación, incertidumbre y volatilidad, contar con modelos de pronóstico confiables permite a entidades públicas, empresas pesqueras y actores de la cadena alimentaria anticipar fluctuaciones en la oferta y la demanda, y planificar de forma estratégica.

El análisis incluirá:

- Visualización exploratoria de la serie histórica.
- Aplicación de al menos tres modelos distintos para pronóstico:
  - **ARIMA:** Modelo estadístico tradicional para series temporales.
  - **Prophet:** Modelo aditivo desarrollado por Meta, adecuado para estacionalidad y tendencias.
  - **Random Forest Regressor:** Algoritmo de Machine Learning no paramétrico que aprovecha variables temporales como mes y año.

Los modelos serán evaluados en un periodo de prueba fuera de muestra mediante métricas como **MAE (Mean Absolute Error)** y **RMSE (Root Mean Squared Error)**. Finalmente, se realizará una interpretación financiera y estratégica de los resultados obtenidos.

---

In [5]:
import numpy as np
import pandas as pd #SIRVE PARA DATAFRAMES
import matplotlib.pyplot as plt #sirve para gráficos
import seaborn as sns #graficos al interior de matplotlib
from statsmodels.tsa.holtwinters import ExponentialSmoothing #para trabajar modelos de econometria

In [2]:
dir()

['ExponentialSmoothing',
 'In',
 'Out',
 '_',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_dh',
 '_i',
 '_i1',
 '_i2',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'exit',
 'get_ipython',
 'np',
 'pd',
 'plt',
 'quit',
 'sns']

🟦 SNIPPET 1 – Cargar el archivo Excel desde INEI

In [17]:
# 📂 Cargar archivo Excel con series pesqueras del INEI

# Ruta del archivo (debe estar en el mismo directorio o haber sido subido en Colab)
file_path = "Series INEI pesca.xlsx"

# Leer el archivo completo sin encabezados
xls = pd.ExcelFile(file_path)
df_full = xls.parse(xls.sheet_names[0], header=None)

/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
